In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv('../data/csv/movies.csv')

In [3]:
ratings = pd.read_csv('../data/csv/ratings.csv')

In [4]:
tags = pd.read_csv('../data/csv/tags.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [8]:
joined = movies.set_index('movieId').join(ratings.set_index('movieId'))
joined = joined[~joined['userId'].isna()] 
joined

,title,genres,userId,rating,timestamp
movieId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
193585,Flint (2017),Drama,184.0,3.5,1.537110e+09


In [9]:
trimmed = joined.reset_index()[['movieId', 'userId', 'rating']]
trimmed

,movieId,userId,rating
0,1,1.0,4.0
1,1,5.0,4.0
2,1,7.0,4.5
3,1,15.0,2.5
4,1,17.0,4.5
...,...,...,...
100831,193581,184.0,4.0
100832,193583,184.0,3.5
100833,193585,184.0,3.5
100834,193587,184.0,3.5


In [93]:
users_set = trimmed['userId'].unique()
users_set.sort()
user_frame = pd.DataFrame(users_set, columns = ['userId'])
user_frame = user_frame.set_index('userId')

movies_set = trimmed['movieId'].unique()
movies_set.sort()
movie_vector_blank = pd.DataFrame([np.zeros(len(movies_set))], columns = movies_set)

user_frame['movie_vect'] = [movie_vector_blank.copy() for i in range(user_frame.shape[0])]

counter = 0
ten_precent = int(trimmed.shape[0] / 10)
for row in trimmed.iterrows():
    counter += 1
    if counter % ten_precent == 0:
        print(counter // ten_precent * 10, '% of the way done')
    row = row[1]
    _userId = row['userId']
    _movieId = row['movieId']
    user_frame.loc[_userId]['movie_vect'][_movieId] = row['rating']

10 % of the way done
20 % of the way done
30 % of the way done
40 % of the way done
50 % of the way done
60 % of the way done
70 % of the way done
80 % of the way done
90 % of the way done
100 % of the way done


In [109]:
user_frame.loc[1]['movie_vect']

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [193]:
user_num = 1
new_user = user_frame.loc[user_num]['movie_vect']
new_frame = user_frame.copy().drop(user_num)

sims = []
for row in new_frame.iterrows():
    userId = row[0]
    row = row[1]
    sims.append((userId, cosine_similarity(row['movie_vect'], new_user)[0][0]))
    
sims.sort(key = lambda x: x[1], reverse = True)
closest_user = int(sims[0][0])
closest_user = new_frame.iloc[closest_user]['movie_vect']

m = (closest_user * new_user).T
top_five = m[m[0] > 0].sort_values(by = 0, ascending = False)[0: 5]

for movie_id in top_five.index:
    top_five.loc[movie_id] = movies.set_index('movieId').loc[movie_id]['title']

In [194]:
top_five

,0
2858,American Beauty (1999)
50,"Usual Suspects, The (1995)"
1196,Star Wars: Episode V - The Empire Strikes Back...
1197,"Princess Bride, The (1987)"
2571,"Matrix, The (1999)"
